<a href="https://colab.research.google.com/github/UW-ERSL/AuTO/blob/main/compliance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports

In [ ]:
# run this first time to clone the directory 
!git clone https://github.com/UW-ERSL/AuTO.git
%cd AuTO/models/

In [2]:
# We begin by importing the necessary libraries
import numpy as np
import jax
import jax.numpy as jnp
from jax import jit, value_and_grad
from jax.ops import index, index_add
import time
from utilfuncs import Mesher, computeLocalElements, computeFilter
from mmaOptimize import optimize
import matplotlib.pyplot as plt

### Mesh

In [3]:
nelx, nely = 60, 30
elemSize = np.array([1., 1.])
mesh = {'nelx':nelx, 'nely':nely, 'elemSize':elemSize,\
        'ndof':2*(nelx+1)*(nely+1), 'numElems':nelx*nely}

### Material

In [4]:
material = {'Emax':1., 'Emin':1e-3, 'nu':0.3, 'penal':3.}

### Filter

In [5]:
filterRadius = 1.5
H, Hs = computeFilter(mesh, filterRadius)
ft = {'type':1, 'H':H, 'Hs':Hs}

### Boundary condition

In [6]:
example = 1
if(example == 1):
    # tip cantilever
    force = np.zeros((mesh['ndof'],1))
    dofs=np.arange(mesh['ndof'])
    fixed = dofs[0:2*(nely+1):1]
    free = jnp.setdiff1d(np.arange(mesh['ndof']),fixed)
    force[2*(nelx+1)*(nely+1)-2*nely+1, 0 ] = -1
    symXAxis = False
    symYAxis = False
elif(example == 2):
    ndof = 2*(nelx+1)*(nely+1)
    force = np.zeros((mesh['ndof'],1))
    dofs=np.arange(mesh['ndof'])
    fixed = dofs[0:2*(nely+1):1]
    free = jnp.setdiff1d(np.arange(mesh['ndof']),fixed)
    force[2*(nelx+1)*(nely+1)- (nely+1), 0 ] = -1
    symXAxis = True
    symYAxis = False
bc = {'force':force, 'fixed':fixed,'free':free,\
          'symXAxis':symXAxis, 'symYAxis':symYAxis}

### Constraints

In [7]:
globalVolumeConstraint = {'isOn':True, 'vf':0.5}

### Optimize

In [8]:
optimizationParams = {'maxIters':200,'minIters':100,'relTol':0.05}
projection = {'isOn':False, 'beta':4, 'c0':0.5}

In [9]:
class ComplianceMinimizer:
    def __init__(self, mesh, bc, material, \
                 globalvolCons, projection):
        self.mesh = mesh
        self.material = material
        self.bc = bc
        M = Mesher()
        self.edofMat, self.idx = M.getMeshStructure(mesh)
        self.K0 = M.getK0(self.material)
        self.globalVolumeConstraint = globalvolCons
        self.objectiveHandle = jit(value_and_grad(self. computeCompliance))
        self.consHandle = self.computeConstraints
        self.numConstraints = 1
        self.projection = projection
    #-----------------------#
    # Code snippet 2.1
    def computeCompliance(self, rho):
        #-----------------------#
        @jit
        # Code snippet 2.9
        def projectionFilter(rho):
            if(self.projection['isOn']):
                v1 = np.tanh(self.projection['c0']*self.projection['beta'])
                nm = v1 + jnp.tanh(self.projection['beta']*(rho-self.projection['c0']))
                dnm = v1 + jnp.tanh(self.projection['beta']*(1.-self.projection['c0']))
                return nm/dnm
            else:
                return rho
        #-----------------------#
        @jit
        # Code snippet 2.2
        def materialModel(rho):
            Y = self.material['Emin'] + \
                (self.material['Emax']-self.material['Emin'])*\
                                (rho+0.01)**self.material['penal']
            return Y
        #-----------------------#
##         @jit
          # Code snippet 2.8
#         def materialModel(rho): # RAMP
#             S = 8. # RAMP param
#             Y = 0.001*self.material['Emax'] +\
#                     self.material['Emax']*(rho/ (1.+S*(1.-rho)) )
#             return Y
#         Y = materialModel(rho)
        #-----------------------#
        @jit
        # Code snippet 2.3
        def assembleK(Y):
            K_asm = jnp.zeros((self.mesh['ndof'], self.mesh['ndof']))
            K_elem = (self.K0.flatten()[np.newaxis]).T 
            K_elem = (K_elem*Y).T.flatten()
            K_asm = jax.ops.index_add(K_asm, self.idx, K_elem)
            return K_asm
        #-----------------------#
        @jit
        # Code snippet 2.4
        def solveKuf(K): 
            u_free = jax.scipy.linalg.solve\
                    (K[self.bc['free'],:][:,self.bc['free']], \
                    self.bc['force'][self.bc['free']], \
                     sym_pos = True, check_finite=False)
            u = jnp.zeros((self.mesh['ndof']))
            u = jax.ops.index_add(u, self.bc['free'],\
                                  u_free.reshape(-1))
            return u
        #-----------------------#
        rho = projectionFilter(rho)
        Y = materialModel(rho)
        K = assembleK(Y)
        u = solveKuf(K)

        return jnp.dot(self.bc['force'].T, u)[0]
    #-----------------------#
    def computeConstraints(self, rho, epoch): 
        @jit
        # Code snippet 2.6
        def computeGlobalVolumeConstraint(rho):
            g = jnp.mean(rho)/self.globalVolumeConstraint['vf'] - 1.
            return g
        # Code snippet 2.7
        c, gradc = value_and_grad(computeGlobalVolumeConstraint)\
                                    (rho);
        c, gradc = c.reshape((1,1)), gradc.reshape((1,-1))
        return c, gradc
    #-----------------------#
    def TO(self, optimizationParams, ft):
        optimize(self.mesh, optimizationParams, ft, \
             self.objectiveHandle, self.consHandle, self.numConstraints)

In [ ]:
Opt = ComplianceMinimizer(mesh, bc, material, \
                globalVolumeConstraint, projection)
Opt.TO(optimizationParams, ft)